In [35]:
from datetime import datetime
import pandas as pd
import json
import math

# my file structure:
# wisd hackathon 2023 (root folder)
# - /games (downloaded games from aws)
# -- /gameid
# - /metadata (downloaded metadata from aws)
# - this notebook
# i hope i wrote this in a way that makes sense!

In [23]:
df = pd.read_csv('joint.csv', index_col=0) # read csv generated by get_pyball.ipynb, use first col as index
df.head()

,gameId,eventType,shotClock,gameClock,wallClock,period,homePlayers,awayPlayers,playerId,pbpId,...,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,TOUCH,24.0,719.76,1652834779899,1,"['82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b', '267d...","['297a0fda-0a03-4003-bf39-aa92b7a730ff', '986b...",561048bb-c412-4d8b-a0bc-27d250a1a431,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,DRIBBLE,24.0,719.76,1652834779899,1,"['82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b', '267d...","['297a0fda-0a03-4003-bf39-aa92b7a730ff', '986b...",561048bb-c412-4d8b-a0bc-27d250a1a431,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,PASS,24.0,719.04,1652834780619,1,"['82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b', '267d...","['297a0fda-0a03-4003-bf39-aa92b7a730ff', '986b...",561048bb-c412-4d8b-a0bc-27d250a1a431,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,TOUCH,23.9,718.96,1652834780699,1,"['82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b', '267d...","['297a0fda-0a03-4003-bf39-aa92b7a730ff', '986b...",297a0fda-0a03-4003-bf39-aa92b7a730ff,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,c17d96b7-4f14-4c4a-9f8b-e96cb63de9ac,DRIBBLE,23.2,718.28,1652834781379,1,"['82a7f31c-e3b4-4105-8d3e-0e4dcde0f73b', '267d...","['297a0fda-0a03-4003-bf39-aa92b7a730ff', '986b...",297a0fda-0a03-4003-bf39-aa92b7a730ff,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
shot_df = df[df['eventType'] =="SHOT"]
print(shot_df.to_string())

                                    gameId eventType  shotClock  gameClock      wallClock  period                                                                                                                                                                                               homePlayers                                                                                                                                                                                               awayPlayers                              playerId  pbpId     GAME_ID  EVENTNUM  EVENTMSGTYPE  EVENTMSGACTIONTYPE  PERIOD WCTIMESTRING PCTIMESTRING                                                    HOMEDESCRIPTION NEUTRALDESCRIPTION                                               VISITORDESCRIPTION      SCORE SCOREMARGIN  PERSON1TYPE  PLAYER1_ID         PLAYER1_NAME  PLAYER1_TEAM_ID PLAYER1_TEAM_CITY PLAYER1_TEAM_NICKNAME PLAYER1_TEAM_ABBREVIATION  PERSON2TYPE  PLAYER2_ID         PLAYER2_NAME  PLAYER2_TEAM_

In [29]:
# wallclock to datetime example

# this wallclock value is taken from game 42100301 (heat vs celtics eastern conf finals game 1)
# that game was played on may 17, 2022 8:30 pm ET

wallclock = 1652834798699  # wallclock is given in ms since the Unix epoch

# fromtimestamp() will return the datetime in your computer's local time (i think)
datetime_obj = datetime.fromtimestamp(wallclock / 1000) # divide by 1000 because wallclock is in ms and fromtimestamp() takes seconds

# using strftime: https://www.programiz.com/python-programming/datetime/strftime
print(datetime_obj.strftime("%m/%d/%Y, %-I:%M:%S %p"))



05/17/2022, 8:46:38 PM


In [28]:
# isolating the frames for each shot

# inner join the shots_df from joint.csv (which is currently only game 00421000301) 
# and the tracking data from that game
tracking_df = pd.read_json("games/0042100301/0042100301_tracking.jsonl", lines=True)
print(len(tracking_df))
print(len(shot_df))

shots_and_tracking_df = pd.merge(
    shot_df, tracking_df, on='wallClock', how='inner', suffixes=('_event', '_tracking'))

print(shots_and_tracking_df.to_string())

96600
179
                                   gameId eventType  shotClock_event  gameClock_event      wallClock  period_event                                                                                                                                                                                         homePlayers_event                                                                                                                                                                                         awayPlayers_event                              playerId  pbpId     GAME_ID  EVENTNUM  EVENTMSGTYPE  EVENTMSGACTIONTYPE  PERIOD WCTIMESTRING PCTIMESTRING                                                    HOMEDESCRIPTION NEUTRALDESCRIPTION                                               VISITORDESCRIPTION      SCORE SCOREMARGIN  PERSON1TYPE  PLAYER1_ID         PLAYER1_NAME  PLAYER1_TEAM_ID PLAYER1_TEAM_CITY PLAYER1_TEAM_NICKNAME PLAYER1_TEAM_ABBREVIATION  PERSON2TYPE  PLAYER2_ID         

In [39]:
def distance3d(p1: list, p2: list) -> float :
     x1,y1,z1 = p1
     x2, y2, z2 = p2
     return math.sqrt(math.pow(x2 - x1, 2) +
                math.pow(y2 - y1, 2) +
                math.pow(z2 - z1, 2)* 1.0)

In [40]:
def away_or_home(row):
    if(row['playerId'] in row['homePlayers_event']):
        return 'home'
    elif (row['playerId'] in row['awayPlayers_event']):
        return 'away'
    else:
        print(f'ERROR: {row["EVENTNUM"]}')
        return None

def get_player_coords(row):
    team = row['shotmaker_team']
    for dict in row[f'{team}Players_tracking']:
      if dict['playerId'] == row['playerId']:
        return dict['xyz']

def get_defender_distances(row):
    team = row['shotmaker_team']
    shotmaker_coord = row['shotmaker_coord']
    dist_arr = []
    if team == 'away':
        for entry in row['homePlayers_tracking']:
            defender_coord = entry['xyz']
            dist_arr.append(distance3d(shotmaker_coord, defender_coord))
    elif team == 'home':
        for entry in row['awayPlayers_tracking']:
            defender_coord = entry['xyz']
            dist_arr.append(distance3d(shotmaker_coord, defender_coord))
    else:
        print(f'ERROR: {row["EVENTNUM"]}')
        return None
    
    return dist_arr


# create new columns on the shots_and_tracking_df
# I decided to find if the shotmaker was on home or away first
# then I found their coordinates
# then I found the distance from all of the defenders
shots_and_tracking_df['shotmaker_team']=shots_and_tracking_df.apply(
    lambda row: away_or_home(row), axis=1)
shots_and_tracking_df['shotmaker_coord'] = shots_and_tracking_df.apply(
    lambda row: get_player_coords(row), axis=1)
shots_and_tracking_df['defense_distance'] = shots_and_tracking_df.apply(
    lambda row: get_defender_distances(row), axis=1)
print(shots_and_tracking_df.to_string())

# Next steps:
# Find avg defense distance?
# Find closest defender distance?
# See if there is any correlation between the above stats and if the shot went in or not

                                   gameId eventType  shotClock_event  gameClock_event      wallClock  period_event                                                                                                                                                                                         homePlayers_event                                                                                                                                                                                         awayPlayers_event                              playerId  pbpId     GAME_ID  EVENTNUM  EVENTMSGTYPE  EVENTMSGACTIONTYPE  PERIOD WCTIMESTRING PCTIMESTRING                                                    HOMEDESCRIPTION NEUTRALDESCRIPTION                                               VISITORDESCRIPTION      SCORE SCOREMARGIN  PERSON1TYPE  PLAYER1_ID         PLAYER1_NAME  PLAYER1_TEAM_ID PLAYER1_TEAM_CITY PLAYER1_TEAM_NICKNAME PLAYER1_TEAM_ABBREVIATION  PERSON2TYPE  PLAYER2_ID         PLAYER2_NA

In [30]:
# creating a column for the coordinates of the shot-taker

# merging home players column and away players column so looking for player is easier (to isolate shot taker coords)
def merge_players(row):
    merged_list = []
    merged_list.extend(row['homePlayers_tracking'])
    merged_list.extend(row['awayPlayers_tracking'])
    return merged_list

shots_and_tracking_df['players'] = shots_and_tracking_df.apply(lambda row: merge_players(row), axis = 1)

# extracting the coordinates of the player that matches the playerId column
def get_player_coords(row):
    for dict in row['players']:
      if dict['playerId'] == row['playerId']:
        return dict['xyz']

# dropping all columns except for the player data and the playerId of the shot taker
coordinates_df = shots_and_tracking_df[['homePlayers_tracking', 'awayPlayers_tracking', 'players', 'playerId']]

coordinates_df['player_coord'] = coordinates_df.apply(lambda row: get_player_coords(row), axis=1)

print(coordinates_df.to_string())

# NEXT STEPS:
# want to isolate the coordinates of the defenders, i.e. the column that does NOT have the shot taker in it
# right now, we have a column of homeplayers, awayplayers, and all the players on the court
# need to figure out a way determine which column the playerId is NOT in and keep that column
# might want to consider separating out which team takes the shot? two different tables?

/var/folders/ds/y506zm_56zv14d0zsmmsnz_c0000gn/T/ipykernel_2281/2964058298.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coordinates_df['player_coord'] = coordinates_df.apply(lambda row: get_player_coords(row), axis=1)


                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           homePlayers_tracking                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         